# Fall 2024 Data Science Track: Week 2 - Data Cleaning Exercise

## Packages, Packages, Packages!

Import *all* the things here! You need the standard stuff: `pandas` and `numpy`.

If you got more stuff you want to use, add them here too. 🙂

In [2]:
# Import here.
import pandas as pd
import numpy 


## Introduction

With the packages out of the way, now you will be working with the following data sets:

* `food_coded.csv`: [Food choices](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv) from Kaggle
* `Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`: [Ask A Manager Salary Survey 2021 (Responses)](https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/view?&gid=1625408792) as *Tab Separated Values (.tsv)* from Google Docs

Each one poses different challenges. But you’ll―of course―overcome them with what you learned in class! 😉

## Food Choices Data Set

### Load the Data

In [7]:
# Load the Food choices data set into a variable (e.g., df_food).

food_data_set_path = '../data/food_coded.csv'


df_food = pd.read_csv(food_data_set_path)

### Explore the Data

How much data did you just load?

In [8]:
# Count by hand. (lol kidding)
len(df_food)

125

What are the columns and their types in this data set?

In [9]:
# Show the column names and their types.
df_food.dtypes


GPA                  object
Gender                int64
breakfast             int64
calories_chicken      int64
calories_day        float64
                     ...   
type_sports          object
veggies_day           int64
vitamins              int64
waffle_calories       int64
weight               object
Length: 61, dtype: object

### Clean the Data

Perhaps we’d like to know more another day, but the team is really interested in just the relationship between calories (`calories_day`) and weight. …and maybe gender.

Can you remove the other columns?

In [3]:
# Remove ‘em.
df_clean = df_food[['Gender','calories_day','weight']]

NameError: name 'df_food' is not defined

What about `NaN`s? How many are there?

In [14]:
# Count ‘em.
df_clean.isna().sum()


Gender           0
calories_day    19
weight           2
dtype: int64

We gotta remove those `NaN`s―the entire row.

In [15]:
# Drop ‘em.
df_clean=df_clean.dropna()


But what about the weird non-numeric values in the column obviously meant for numeric data?

Notice the data type of that column from when you got the types of all the columns?

If only we could convert the column to a numeric type and drop the rows with invalid values. 🤔

In [16]:
# Fix that.
df_clean['weight'] = pd.to_numeric(df_clean['weight'], errors='coerce')


Now this data seems reasonably clean for our purposes! 😁

Let’s save it somewhere to be shipped off to another teammate. 💾

In [4]:
# Savey save!
df_clean.to_csv('../data/CB_food_cleaned.csv', index=False)


NameError: name 'df_clean' is not defined

## Ask a Manager Salary Survey 2021 (Responses) Data Set

### Load the Data

In [15]:
df = pd.read_csv('../data/AskAManager_SalarySurvey2021_Responses.csv', sep='\t', on_bad_lines='warn')

Was that hard? 🙃

#### rename the file to something that is better for all systems.  
* No spaces in filename (can use '_')
* all lower case

### Explore

You know the drill.

How much data did you just load?

In [18]:
# Count by hand. I’m dead serious.
len(df)



28062

What are the columns and their types?

In [19]:
# Show the column names and their types.
df.dtypes


Timestamp                                                                                                                                                                                                                                object
How old are you?                                                                                                                                                                                                                         object
What industry do you work in?                                                                                                                                                                                                            object
Job title                                                                                                                                                                                                                                object
If your job title needs additional conte

Oh… Ugh! Give these columns easier names to work with first. 🙄

In [20]:
# Rename ‘em.
# Non-binding suggestions: timestamp, age, industry, title, title_context, salary, additional_compensation, currency, other_currency, salary_context, country, state, city, total_yoe, field_yoe, highest_education_completed	gender, race
df.columns = ['timestamp', 'age', 'industry', 'title', 'title_context', 'salary', 'additional_compensation', 'currency', 'other_currency', 'salary_context', 'country', 'state', 'city', 'total_yoe', 'field_yoe', 'highest_education_completed', 'gender', 'race']


It’s a lot, and that should not have been easy. 😏

You’re going to have a gander at the computing/tech subset first because thats *your* industry. But first, what value corresponds to that `industry`?

In [21]:
# List the unique industries and a count of their instances.
print(df['industry'].value_counts())


industry
Computing or Tech                        4699
Education (Higher Education)             2464
Nonprofits                               2419
Health care                              1896
Government and Public Administration     1889
                                         ... 
Warehousing                                 1
Education (Early Childhood Education)       1
SAAS                                        1
Health and Safety                           1
Aerospace Manufacturing                     1
Name: count, Length: 1219, dtype: int64


That value among the top 5 is what you’re looking for innit? Filter out all the rows not in that industry and save it into a new dataframe. 

In [29]:
# Filtery filter. (Save it to a new variable, df_salary_tech.)
df_salary_tech = df[df['industry'] == 'Computing or Tech']

Do a sanity check to make sure that the only values you kept are the one you are filtered for.  

In [34]:
# Sanity Check 
print(df_salary_tech['industry'].unique())


['Computing or Tech']


We are very interested in salary figures. But how many dollars 💵 is a euro 💶 or a pound 💷? That sounds like a problem for another day. 🫠

For now, let’s just look at U.S. dollars (`'USD'`).

In [39]:
# Filtery filter for just the jobs that pay in USD!
df_salary_tech_usd = df_salary_tech[df_salary_tech['currency'] == 'USD']
print(df_salary_tech_usd['currency'].unique())

['USD']


What we really want know is how each U.S. city pays in tech. What value in `country` represents the United States of America?

In [40]:
# We did filter for USD, so if we do a count of each unique country in descending count order, the relevant value(s) should show up at the top.
df_salary_tech_usd['country'].value_counts()


country
United States               1576
USA                         1222
US                           412
U.S.                         108
United States of America      90
                            ... 
Ghana                          1
Nigeria                        1
ss                             1
Nigeria                        1
Burma                          1
Name: count, Length: 76, dtype: int64

### Clean the Data

Well, we can’t get our answers with what we currently have, so you’ll have to make some changes.

Let’s not worry about anything below the first 5 values for now. Convert the top 5 to a single canonical value―say, `'US'`, which is nice and short.

In [45]:
# Replace them all with 'US'.
df_salary_tech_usd.loc[:,'country'] = df_salary_tech_usd['country'].replace({
    'United States': 'US',
    'USA': 'US',
    'United States of America': 'US',
    'U.S.': 'US'
})


Have a look at the count of each unique country again now.

In [46]:
# Count again.
df_salary_tech_usd['country'].value_counts()


country
US                3408
United States       68
Usa                 59
USA                 56
usa                 28
                  ... 
Ghana                1
Nigeria              1
ss                   1
Nigeria              1
Burma                1
Name: count, Length: 72, dtype: int64

Did you notice anything interesting?

In [57]:
# BONUS CREDIT: resolve [most of] those anomalous cases too without exhaustively taking every variant literally into account.

df_salary_tech_usd.loc[:,'country'] = (
    df_salary_tech_usd['country']
    .str.strip()
    .str.replace('.', '', regex=False)
    .str.title()
 )

In [58]:

# BONUS CREDIT: if you’ve resolved it, let’s see how well you did by counting the number of instances of each unique value.
df_salary_tech_usd['country'].value_counts()


country
Us                          3436
Usa                          155
United States                108
United States Of America      11
Israel                         5
Canada                         5
Australia                      2
United State Of America        2
Unitedstates                   2
United Kingdom                 2
France                         2
Poland                         2
Brazil                         2
Singapore                      2
Spain                          2
India                          2
Unite States                   2
New Zealand                    2
Denmark                        2
Nigeria                        2
Danmark                        1
Uniyed States                  1
America                        1
Puerto Rico                    1
United State                   1
Italy                          1
International                  1
Cuba                           1
Uruguay                        1
Isa                            1
Un

It’s looking good so far. Let’s find out the minimum, mean, and maximum (in that order) salary by state, sorted by the mean in descending order.

In [ ]:
# Find the minimum, mean, and maximum salary in USD by U.S. state.
df_us = df_salary_tech_usd[df_salary_tech_usd['country'] == 'Us']
summary = (
    df_us.groupby('state')['salary']
    .agg(['min', 'mean', 'max'])
    .sort_values('mean', ascending=False)
)
summary

,min,mean,max
state,,,
Florida,28800.0,229567.65000,2600000.0
"California, Oregon",200000.0,200000.00000,200000.0
"California, Colorado",176000.0,176000.00000,176000.0
Connecticut,68000.0,170137.50000,270000.0
California,0.0,155776.69375,520000.0
...,...,...,...
Mississippi,NaN,NaN,NaN
"New York, Texas",NaN,NaN,NaN
North Dakota,NaN,NaN,NaN


Well, pooh! We forgot that `salary` isn’t numeric. Something wrong must be fixed.

In [70]:
# Fix it.
df_salary_tech_usd['salary'] = pd.to_numeric(df_salary_tech_usd['salary'], errors='coerce')



/tmp/ipykernel_1843/1542867887.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_salary_tech_usd['salary'] = pd.to_numeric(df_salary_tech_usd['salary'], errors='coerce')


Let’s try that again.

In [71]:
# Try it again. Yeah!
df_us = df_salary_tech_usd[df_salary_tech_usd['country'] == 'Us']
summary = (
    df_us.groupby('state')['salary']
    .agg(['min', 'mean', 'max'])
    .sort_values('mean', ascending=False)
)
summary


,min,mean,max
state,,,
Florida,28800.0,229567.65000,2600000.0
"California, Oregon",200000.0,200000.00000,200000.0
"California, Colorado",176000.0,176000.00000,176000.0
Connecticut,68000.0,170137.50000,270000.0
California,0.0,155776.69375,520000.0
...,...,...,...
Mississippi,NaN,NaN,NaN
"New York, Texas",NaN,NaN,NaN
North Dakota,NaN,NaN,NaN


That did the trick! Now let’s narrow this to data 2021 and 2022 just because (lel). *(Hint: that timestamp column may not be a temporal type right now.)*

In [73]:
# Filter the data to within 2021, 2022, or 2023, saving the DataFrame to a new variable, and generate the summary again.
df_salary_tech_usd['timestamp'] = pd.to_datetime(df_salary_tech_usd['timestamp'], errors='coerce')


df_recent = df_salary_tech_usd[
    df_salary_tech_usd['timestamp'].notna() &
    df_salary_tech_usd['timestamp'].dt.year.isin([2021, 2022, 2023]) &
    (df_salary_tech_usd['country'] == 'Us')
]
summary_recent = (
    df_recent.groupby('state')['salary']
    .agg(['min', 'mean', 'max'])
    .sort_values('mean', ascending=False)
)
summary_recent


/tmp/ipykernel_1843/520620466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_salary_tech_usd['timestamp'] = pd.to_datetime(df_salary_tech_usd['timestamp'], errors='coerce')


,min,mean,max
state,,,
"California, Oregon",200000.0,200000.000000,200000.0
"California, Colorado",176000.0,176000.000000,176000.0
Connecticut,68000.0,170137.500000,270000.0
California,0.0,156316.169811,520000.0
New York,14000.0,148213.564356,590000.0
...,...,...,...
"New York, Texas",NaN,NaN,NaN
North Dakota,NaN,NaN,NaN
South Carolina,NaN,NaN,NaN


## Bonus

Clearly, we do not have enough data to produce useful figures for the level of specificity you’ve now reached. What do you notice about Delaware and West Virginia?

Let’s back out a bit and return to `df_salary` (which was the loaded data with renamed columns but *sans* filtering).

### Bonus #0

Apply the same steps as before to `df_salary`, but do not filter for any specific industry. Do perform the other data cleaning stuff, and get to a point where you can generate the minimum, mean, and maximum by state.

### Bonus #1

This time, format the table output nicely (*$12,345.00*) without modifying the values in the `DataFrame`. That is, `df_salary` should be identical before versus after running your code.

(*Hint: if you run into an error about `jinja2` perhaps you need to `pip install` something.*)

### Bonus #2

Filter out the non-single-states (e.g., `'California, Colorado'`) in the most elegant way possible (i.e., *not* by blacklisting all the bad values).

### Bonus #3

Show the quantiles instead of just minimum, mean, and maximum―say 0%, 5%, 25%, 50%, 75%, 95%, and 100%. Outliers may be deceiving.

Sort by whatever interests you―like say the *50th* percentile.

And throw in a count by state too. It would be interesting to know how many data points contribute to the figures for each state. (*Hint: your nice formatting from Bonus #1 might not work this time around.* 😜)